Data Ingestion stage
 - first updated config.yaml--> params.yaml files
 

In [12]:
import os
os.chdir("../")
# now updating for entity

from collections import namedtuple
import os


DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_url",
    "local_data_file",
    "unzip_dir"
    ])

#or you can follow datclass 
from pathlib import Path 
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [14]:
#write-> configurationmanager
#import constants

from src.CNNClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.CNNClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file =  config.local_data_file,
            unzip_dir = config.unzip_dir
            )
        return data_ingestion_config

In [17]:
#write-components to define separate job of project

import os
import urllib.request as request
from zipfile import ZipFile

# data Ingestion 
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    #def function to download data from url/any other sources(code will be chnaged)
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(url = self.config.source_url,
                                filename = self.config.local_data_file
                                )
            
    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith('.jpg') and ("Cat" in f or "Dog" in f)]
    

    def _preprocess(self,zf: ZipFile,f: str, working_dir: str ):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)

        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)

    #def function to unzip and clean data
    def unzip_and_clean(self):
        with ZipFile(file = self.config.local_data_file,mode="r") as zf:
            list_of_files = zf.namelist()
            
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)

In [18]:
#pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2025-10-04 12:45:21,345: INFO: common]: yaml file: configs\config.yaml loaded successfully]
[2025-10-04 12:45:21,348: INFO: common]: yaml file: params.yaml loaded successfully]
[2025-10-04 12:45:21,351: INFO: common]: Directory created at: artifacts]
[2025-10-04 12:45:21,351: INFO: common]: Directory created at: artifacts/data_ingestion]


In [13]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project'